In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

## GIVEN CONSTANTS
R = 89.17           # Total blade radius [m]
N = 3               # Number of blades
P_rated = 10e7      # Rated power [W]
cut_in = 4.0        # Cut-in wind speed [m/s]
cut_out = 25.0      # Cut-out wind speed [m/s]
rho = 1.225         # Air density [kg/m^3]
P_mech = 10.64e6    # Mechanical power at rated conditions [W]
A = np.pi * R**2    # Swept area [m^2]


# GLOBAL PARAMETER SWEEP GRID
LAMBDA_GRID = np.linspace(5.0, 10.0, 30)
THETA_GRID  = np.linspace(-3.0, 4.0, 30)

##----------------------------DATA FILES----------------------------------

#PATH
BASE_DIR = "/Users/mariafernandeztello/Downloads/AEROEXAM/BEM"    #Data directory

#BLADE FILE --- Blade data from assignment 1. Airfoil data ( chord, twist, t/c) is included here
blade_path = os.path.join(BASE_DIR, "bladedat.txt")
blade = np.loadtxt(blade_path)
r = blade[:, 0]                         #Radial distribution
c = blade[:, 1]                         #Local chord
beta = blade[:, 2]                      #Local Twist
t_c_ratio = blade[:, 3]                 #Local twist to chord ratio

#BLADE STRUCTURE FILE --- Blade structure data from Static Beam exercise
struc_path = os.path.join(BASE_DIR, "bladestruc.txt")
blade_struc = np.loadtxt(struc_path)
pitch_struc = blade_struc[:, 1]          #Structural pitch [deg]
mass_dist = blade_struc[:, 2]            #Mass distribution [kg/m]
EI_1 = blade_struc[:, 3]                 #Torsional stiffnesses [Nm^2]
EI_2 = blade_struc[:, 4]                 #Bending stiffnesses [Nm^2]
twist_struc = blade_struc[:, 5]          #Structural twist [deg]

#AIRFOIL POLARS --- Airfoil data files for different thickness profiles, used for double interpolation
AIRFOIL_FILES = {
    100.0: "cylinder.txt",
     60.0: "FFA-W3-600.txt",
     48.0: "FFA-W3-480.txt",
     36.0: "FFA-W3-360.txt",
     30.1: "FFA-W3-301.txt",
     24.1: "FFA-W3-241.txt",
}

#----------------------------FUNCTIONS TO INTERPOLATE DATA FILES -------------------------------------

#Values of t_c ratio for each file --- Used to select the appropriate airfoil data during double interpolation
def load_airfoils():
    def read_polar(path):                                               #Function to load airfoil files
        tbl = np.loadtxt(path, ndmin=2)                                 #Reads each Airfoil file and creates a 2D array -- [alpha, Cl, Cd]
        order = np.argsort(tbl[:, 0])                                   #Orders the table, returning it with ascending alpha order
        return tbl[order, 0], tbl[order, 1], tbl[order, 2]  # alpha, Cl, Cd

    # sort by thickness
    items = sorted(AIRFOIL_FILES.items(), key=lambda kv: kv[0])         # AIRFOIL_FILES.items: Gives pairs item, filename;  Sorted... : Sorts the files by thickness ( 24.1 --> 100)

    thick_prof = np.array([tc for tc, _ in items], dtype=float)         #Extracts the thickness to a NumPy array for thickness interpolation
    aoa_cols   = []                                                     #Initialize storing lists to store alpha, cl and cd for each thickness
    cl_cols    = []
    cd_cols    = []

    for tc, fname in items:                                             #For each file, load each column into the corresponding array
        alpha, cl, cd = read_polar(os.path.join(BASE_DIR, fname))       #Values read from function
        aoa_cols.append(alpha)                                          #Append to angle of attack array
        cl_cols.append(cl)                                              #Appned to cl array
        cd_cols.append(cd)                                              #Append to cd array

    return thick_prof, aoa_cols, cl_cols, cd_cols

##DOUBLE INTERPOLATION: first in alpha, then in thickness --- Returns Cl and Cd for given alpha (deg) and t/c (%)
thick_prof, aoa_cols, cl_cols, cd_cols = load_airfoils()

def cl_cd_double_interp(alpha_deg: float, tc_percent: float):

    clthick = np.empty(len(thick_prof), dtype=float)                    #Create empty arrays for Cl and Cd for each thickness
    cdthick = np.empty(len(thick_prof), dtype=float)
    a = float(alpha_deg)                                                #Angle of attack
    for k in range(len(thick_prof)):                                    #For each thickness, interpolate the Cl and Cd for the selected angle of attack
        clthick[k] = np.interp(a, aoa_cols[k], cl_cols[k])
        cdthick[k] = np.interp(a, aoa_cols[k], cd_cols[k])

    t = float(np.clip(tc_percent, thick_prof.min(), thick_prof.max()))  #Selected thickness
    clift = np.interp(t, thick_prof, clthick)                           #For the selected thickness, interpolate Cl and Cd across thickness dimension to find just the desired output
    cdrag = np.interp(t, thick_prof, cdthick)
    return clift, cdrag


#--------------------FUNCTIONS FOR INTERMEDIATE CALCULATIONS -------------------------

##PRANDT LOSSES --- Calculates Prandtl tip loss factor F
def prandtl_tip_loss(B, r, R, phi, root_cut=0.0):
    F=(2/np.pi)*np.acos(np.exp(-(B/2)*((R-r)/(r*np.abs(np.sin(phi))))))
    return np.clip(F, 1e-4, 1.0)

#EQ2 --- Function to calculate axial induction factor a from CT and F using Glauert's empirical relation
def a_eq2(CT, F):
    x = CT / F
    return 0.246*x + 0.0586*(x**2) + 0.0883*(x**3)


#----------------------------------PRINCIPAL FUNCTIONS---------------------------

#BEM ELEMENT --- BEM element solver for a single radial station. Outputs a, a', flow angle (phi), F, Cn, Ct, Vrel, pn, pt, CT, sigma
def bem_element(V0, omega, r, c, beta_deg, pitch_deg, tc_percent, B, R, rho, cl_cd_fn,
    relax=0.10, eps=1e-5, max_iter=1000, eq="eq1"):

    sigma = (B * c) / (2 * np.pi * r)                                       #Solidity computation
    theta = np.deg2rad(beta_deg + pitch_deg)                                #Local pitch angle

    a = 0.0
    ap = 0.0

    it = 0
    while True:                                                             #While convergence is not reached
        it += 1
        #step1
        phi = np.arctan2((1 - a) * V0, (1 + ap) * omega * r)                #Flow angle computation
        #step2
        alpha_deg = np.rad2deg(phi - theta)                                 #Angle of attack
        cl, cd = cl_cd_fn(alpha_deg, tc_percent)                            # Cl and cd --> From function
        #step3
        Cn = cl * np.cos(phi) + cd * np.sin(phi)                            #Normal coefficient
        Ct = cl * np.sin(phi) - cd * np.cos(phi)                            #Tangencial coefficient
        #step4 -- Compute Thrust Coefficient
        CT = ((1 - a) ** 2) * Cn * sigma / ((np.sin(phi))**2)
        #Tip-loss factor
        F = prandtl_tip_loss(B, r, R, phi)                                  #Tip-Loss factor --> From function

        #a* and a'*
        if a <= 1/3:
            a_star = ((sigma * Cn) / (4 * F * (np.sin(phi))**2)) * (1 - a)
        else:
            if eq == "eq2":
                a_star = a_eq2(CT, F)
            else:
                a_star = CT / (4 * F * (1 - 0.25 * (5 - 3 * a) * a))

        ap_star = (sigma * Ct) / (4 * F * np.sin(phi) * np.cos(phi)) * (1 + ap)         #a'*

        #New values of a and a'
        a_new = relax * a_star + (1.0 - relax) * a
        ap_new = relax * ap_star + (1.0 - relax) * ap

        if max(abs(a_new - a), abs(ap_new - ap)) < eps or it >= max_iter:           #Convergence check
            a, ap = a_new, ap_new
            break

        a, ap = a_new, ap_new

    Vrel = ((1 + ap) * omega * r) / np.cos(phi)                                     #Relative veolcity [m/s]
    pn = 0.5 * rho * Vrel**2 * c * Cn                                               # normal force per unit span [N/m]
    pt = 0.5 * rho * Vrel**2 * c * Ct                                               #Tangencial force per unit span [N/m]

    return {"a": a, "ap": ap, "phi": phi, "F": F,"Cn": Cn, "Ct": Ct, "Vrel": Vrel, "pn": pn, "pt": pt,
        "CT": CT, "sigma": sigma}


#CP, CT --- Function to compute final values of Cp and CT over ranges of lambda and theta, outputs Cp and CT matrices, maximum Cp and corresponding lambda and theta
def compute_finalvalues(eq="eq1", V0=10.0):                             #Arbitrary value of V0
    global rho

    lambdas = LAMBDA_GRID                                                   #Tip-speed ratio to find optimum
    thetas_deg = THETA_GRID                                                 #Pitch to find optimum

    r_nodes = r                                                             #Function uses variables to be able to change them without affecting global values.
    c_nodes = c
    beta_nodes = beta
    tc_nodes = t_c_ratio

    pt_dist = np.zeros(len(r_nodes))
    pn_dist = np.zeros(len(r_nodes))

    # empty vectors to fill with the loops and iterations
    Cp = np.zeros((len(thetas_deg), len(lambdas)))
    CT = np.zeros_like(Cp)

    # loop over TSR and pitch
    for j, lam in enumerate(lambdas):                                   #For each tip-speed ration (lambda)
        omega = lam * V0 / R                                            #Calculated rotational speed

        for i, theta_p in enumerate(thetas_deg):                        #For each pitch
            Q = 0.0 #Torque
            T = 0.0 #Thrust

            # integrate along the blade (per-span -> ring forces)
            for k in range(len(r_nodes)):
                ri = r_nodes[k]
                ci = c_nodes[k]
                betai = beta_nodes[k]
                tci = tc_nodes[k]

                #Get the BEM values for the selected conditions -- Runs BEM per iteration
                out = bem_element(V0=V0, omega=omega, r=ri, c=ci, beta_deg=betai, pitch_deg=theta_p,tc_percent=tci, B=N, R=R, rho=rho, cl_cd_fn=cl_cd_double_interp, eq=eq)

                pn = out["pn"]                                          #Gets Pn and Pt from BEM function
                pt = out["pt"]

                # tip load to zero
                if k == len(r_nodes) - 1:
                    pn = 0.0
                    pt = 0.0

                pn_dist[k] = pn
                pt_dist[k] = pt

            T = N * np.trapezoid(pn_dist,r_nodes)                       #Thrust
            Q = N * np.trapezoid(pt_dist*r_nodes,r_nodes)               #Torque

            P = omega * Q                                               #Power

            Cp[i, j] = P / (0.5 * rho * A * V0**3)                      #Power Coefficient
            CT[i, j] = T / (0.5 * rho * A * V0**2)                      #Thrust Coefficient

    # Find the 2D index (row = theta index, col = lambda index) of the maximum Cp value.
    # np.argmax(Cp) gives the flat index of the maximum cp, and unravel_index maps it back to (row, col).
    idx = np.unravel_index(np.argmax(Cp), Cp.shape)
    Cp_max = Cp[idx]
    theta_star = thetas_deg[idx[0]]
    lambda_star = lambdas[idx[1]]

    return Cp, CT, Cp_max, lambda_star, theta_star


def compute_rated (Cp_max, lambda_star, A , Eq = "Eq"):
    global rho

    V0 = np.linspace(cut_in, cut_out, 200)                     #Define a linspace for V0 -- Could be done in variables section on top

    V0_rated = (P_mech / (0.5 * rho * A * Cp_max)) ** (1 / 3)       #Rated Wind speed
    omega_max = (lambda_star * V0_rated) / R                        #Maximum rotor speed -- For above rated conditions

    omega_unclipped = lambda_star * V0 / R                          #Array for rotor speed
    omega = np.minimum(omega_unclipped, omega_max)                  #Array for rotor speed, capping at rated conditions (For plots)

    return V0, V0_rated, omega_max, omega, omega_unclipped


#BEM for complete radius. Function to compute BEM along the complete radius of the blade (used for question 3 in Assignment 1). Outputs power P, thrust T, Cp and Ct
def bem_complete_radius (v, omega, theta_p, eq="eq1"):

    pn_dist = np.zeros(len(r))
    pt_dist = np.zeros(len(r))

    for k in range(len(r)):
        out = bem_element(V0=v, omega=omega, r=r[k], c=c[k], beta_deg=beta[k], pitch_deg=theta_p,
            tc_percent=t_c_ratio[k], B=N, R=R, rho=rho, cl_cd_fn=cl_cd_double_interp, eq=eq)
        pn, pt = out["pn"], out["pt"]
        if k == len(r) - 1:
            pn = 0.0
            pt = 0.0
        pn_dist[k] = pn
        pt_dist[k] = pt

    T = N * np.trapezoid(pn_dist, r)
    Q = N * np.trapezoid(pt_dist * r, r)
    P = omega * Q

    Cp = P / (0.5 * rho * A * v**3)
    Ct = T / (0.5 * rho * A * v**2)
    return P, T, Cp, Ct


#PITCH CONTROL --- Function to compute pitch above and below rated conditions, for all wind speeds (used for question 3 in Assignment 1). Outputs arrays of pitch, omega, lambda, power, thrust, Cp and Ct over wind speed range
def pitch_control(lambda_star, V0_rated, theta_star, omega_max, eq="eq1", mode="feather"):

    V0 = np.linspace(cut_in, cut_out, 70)                   #Define a linspace for V0 -- Could be done in variables section on top

    theta_V0 = np.zeros_like(V0)
    omega_V0 = np.zeros_like(V0)
    lambda_V0 = np.zeros_like(V0)
    Cp_V0    = np.zeros_like(V0)
    Ct_V0    = np.zeros_like(V0)
    P_V0     = np.zeros_like(V0)
    T_V0     = np.zeros_like(V0)

    P_tol = 0.005 * P_mech                                      #Tolerance for mechanical power
    max_it = 100                                                #Maximum number of iterations
    theta_min = -20                                             #Stablish range to find pitch for each conditions
    theta_max = 30
    theta_prev = theta_star                                     #Initialize pitch with below rated pitch

    for k , v in enumerate(V0):                                 #For each wind speed

        #Operating below rated velocity --- Values obtained straight from BEM, no interpolation necessary
        if v<V0_rated:
            omega_V0[k] = (lambda_star * v)/R
            lambda_V0 [k] =  lambda_star
            theta_V0[k] = theta_star
            P_V0[k], T_V0[k], Cp_V0[k], Ct_V0[k] = bem_complete_radius(v, omega_V0[k], theta_V0[k], eq=eq)

        #Operating above rated conditions --- Interpolation to find optimum pitch
        else:

            omega_V0[k] = omega_max                     #Cap rotor speed to rated

            lambda_V0[k] = omega_V0[k] * R / v          #Recalculate tip-speed ratio from maximum rotor speed and wind speed

            # Evaluate at previous pitch --- Get BEM results for new wind speed with previous pitch

            P0, T0, Cp0, Ct0 = bem_complete_radius(v, omega_V0[k], theta_prev, eq=eq)

            f0 = P_mech - P0                            #Power error -- Gets difference in Mechanical power and power obtained from BEM

            # Build a bounded window whose direction depends on f0 and mode

            span = 20.0
            #This part sets the bound for search of theta. Span is just how far the code should look for.
            if mode == "feather":

                # feather: if P too low -> decrease θ; if P too high -> increase θ

                if f0 > 0:  # need more power

                    a = max(theta_min, theta_prev - span);
                    b = theta_prev

                else:  # need less power

                    a = theta_prev;
                    b = min(theta_max, theta_prev + span)

            else:  # "stall"

                # stall: if P too low -> increase θ; if P too high -> decrease θ

                if f0 > 0:  # need more power

                    a = theta_prev;
                    b = min(theta_max, theta_prev + span)

                else:  # need less power

                    a = max(theta_min, theta_prev - span);
                    b = theta_prev

            # Ensure ordering and bounds -- Ensures that the bounds for the search are always:  a--> lower bound, b--> upper bound
            if a > b:
                a, b = b, a

            a = np.clip(a, theta_min, theta_max);
            b = np.clip(b, theta_min, theta_max)

            #Evaluate power at interval endpoints -- Evaluates power at bounds a,b. Obtains power errors for those bounds
            Pa, Ta, Cpa, Cta = bem_complete_radius(v, omega_V0[k], a, eq=eq)
            fa = P_mech - Pa
            Pb, Tb, Cpb, Ctb = bem_complete_radius(v, omega_V0[k], b, eq=eq)
            fb = P_mech - Pb

            # Bracketing expansion Loop: If no sign change, try expanding within limits a few times
            #The desired bracket makes fa and fb of different sign --> function crosses zero between a and b
            expand = 0

            while fa * fb > 0 and expand < 4 and (a > theta_min or b < theta_max):

                if f0 > 0:  # we need more power -> expand toward +θ

                    b = min(b + 5.0, theta_max)         #Moves upper bound 5º upward (limited by theta max)

                    Pb, *_ = bem_complete_radius(v, omega_V0[k], b, eq=eq);  #Calculates new Pb
                    fb = P_mech - Pb                            #Calculates new fb

                else:  # we need less power -> expand toward −θ

                    a = max(a - 5.0, theta_min)         #Moves lower bound 5º downward (limit by theta min)

                    Pa, *_ = bem_complete_radius(v, omega_V0[k], a, eq=eq);
                    fa = P_mech - Pa

                expand += 1

            # If still no bracket, pick the closest boundary and move on --> Pick the endpoint that gives the smaller error

            if fa * fb > 0:
                if abs(fa) <= abs(fb):
                    theta_V0[k] = a
                    P_V0[k], T_V0[k], Cp_V0[k], Ct_V0[k] = Pa, Ta, Cpa, Cta
                else:
                    theta_V0[k] = b
                    P_V0[k], T_V0[k], Cp_V0[k], Ct_V0[k] = Pb, Tb, Cpb, Ctb
                theta_prev = theta_V0[k]
                continue

            # Bisection inside [a,b] until |Perr| ≤ tol

            it = 0;

            while it < max_it:

                th = 0.5 * (a + b)                                          #Compute midpoint of the bracket

                Pm, Tm, Cpm, Ctm = bem_complete_radius(v, omega_V0[k], th, eq=eq)        #Evaluate Pm

                fm = P_mech - Pm

                if abs(fm) <= P_tol:                                        #If calculated theta is within tolerance -- accepted
                    theta_V0[k] = th

                    P_V0[k], T_V0[k], Cp_V0[k], Ct_V0[k] = Pm, Tm, Cpm, Ctm

                    break
                #If not converged:
                if fa * fm <= 0:                                            #Sign change is between a and theta, so b is moved to theta to recompute
                else:                                                       #Sign change is between theta and b, so moved a to th to recompute
                    a, fa = th, fm

                it += 1

            else:                                                           #Only runs if the while does not converge -- chooses midpoint of a and b to be theta and moves on -- close enough

                theta_V0[k] = 0.5 * (a + b)

                P_V0[k], T_V0[k], Cp_V0[k], Ct_V0[k] = bem_complete_radius(v, omega_V0[k], theta_V0[k], eq=eq)

            theta_prev = theta_V0[k]                                        #Updates pitch for next wind speed

    return {"V0": V0, "theta_V0": theta_V0,"omega_V0": omega_V0,"lambda_V0": lambda_V0, "P_V0": P_V0,"T_V0": T_V0,"Cp_V0": Cp_V0,"Ct_V0": Ct_V0,"mode": mode,"eq": eq,
    }

#LOAD CALCULATOR --- For an imput array of wind speeds, calculates loads at the complete blade. Outputs Wind speed selected, radius distribution, rotation speed for each wind speed, pitch per wind speed, pn and pt per windspeed
def compute_spanwise_loads(V0_array, lambda_star, theta_star, V0_rated, omega_max, eq="eq1", mode="feather"):

    V0_array = np.asarray(V0_array, dtype=float)
    r_range  = r

    # get Q3 schedule once (same as q4_solve and deflection_calc)
    q3_out = pitch_control(lambda_star, V0_rated, theta_star, omega_max, eq=eq, mode=mode)
    Vgrid = q3_out["V0"]                        # Wind speed greed used in pitch controller
    thetas_v = q3_out["theta_V0"]               # Complete pitch grid computed in pitch control

    omega_all = np.zeros_like(V0_array)
    theta_all = np.zeros_like(V0_array)
    pn_all    = np.zeros((len(V0_array), len(r_range)))
    pt_all    = np.zeros((len(V0_array), len(r_range)))

    for k, v in enumerate(V0_array):
        if v < V0_rated:
            omega_all[k] = lambda_star * v / R
            theta        = theta_star
        else:
            omega_all[k] = omega_max
            theta        = float(np.interp(v, Vgrid, thetas_v))

        theta_all[k] = theta

        for i, ri in enumerate(r_range):
            ci, betai, tci = c[i], beta[i], t_c_ratio[i]
            out = bem_element(v, omega_all[k], ri, ci, betai, theta, tci,
                              N, R, rho, cl_cd_double_interp, eq=eq)
            pn, pt = out["pn"], out["pt"]
            if i == len(r_range) - 1:
                pn, pt = 0.0, 0.0
            pn_all[k, i] = pn
            pt_all[k, i] = pt

    return {
        "V0": V0_array,
        "r": r_range,
        "omega": omega_all,
        "theta": theta_all,
        "pn": pn_all,
        "pt": pt_all,
    }

#LOADS AT SELECTED WIND SPEEDS --- Function to compute loads at selected wind speeds (used for question 4 in Assignment 1). Outputs distributions of dT/dr and dQ/dr along the blade for selected wind speeds, as well as total T and Q
def loads_selected(lambda_star, theta_star, V0_rated, omega_max, eq="eq1"):
    V0_selected = np.array([5, 9, 11, 20], dtype=float)

    loads = compute_spanwise_loads(V0_selected, lambda_star, theta_star,V0_rated, omega_max, eq=eq, mode="feather")

    V0s = loads["V0"]
    r   = loads["r"]
    pn_all = loads["pn"]
    pt_all = loads["pt"]
    omega_selected = loads["omega"]
    pitch_selected = loads["theta"]

    dTdr_all = N * pn_all
    dQdr_all = N * pt_all
    T_all = N * np.trapezoid(pn_all, r, axis=1)
    Q_all = N * np.trapezoid(pt_all * r[None, :], r, axis=1)

    return {
        "V0_selected": V0s, "r": r,
        "dTdr_all": dTdr_all, "dQdr_all": dQdr_all,
        "T_all": T_all, "Q_all": Q_all,
        "omega_selected": omega_selected,
        "lambda_star": lambda_star, "theta_star": theta_star,
        "eq": eq, "pitch_selected": pitch_selected
    }


# DEFLECTIONS --- Deflections calculator for selected wind speeds. Returns wind speeds, radius distribution, Y and Z deflections, and ThetaY and ThetaZ angles
def deflection_calc(lambda_star, theta_star, V0_rated, omega_max,
                    pitch_struc, mass_dist, EI_1, EI_2, twist_struc,
                    eq='eq1'):

    V0_selected = np.array([6, 11, 20], dtype=float)     # selected wind speeds
    r_range = r                                          # radius distribution

    nV = len(V0_selected)
    Nr = len(r_range)

    # Output arrays
    uy_all = np.zeros((nV, Nr))                          # edgewise deflection
    uz_all = np.zeros((nV, Nr))                          # flapwise deflection
    rota_y_all = np.zeros((nV, Nr))                      # rotation about y
    rota_z_all = np.zeros((nV, Nr))                      # rotation about z

    # Get Q3 operating schedule once (pitch + omega for each wind speed)
    mode_q3 = "feather"  # set to "stall" if needed
    loads = compute_spanwise_loads(V0_selected, lambda_star, theta_star,
                                   V0_rated, omega_max, eq=eq, mode=mode_q3)

    # Loop through selected wind speeds
    for k, v in enumerate(V0_selected):

        # Retrieve aerodynamic loads and pitch for this wind speed
        pn_dist = loads["pn"][k, :]
        pt_dist = loads["pt"][k, :]
        theta = loads["theta"][k]

        # Reset internal force arrays for this V0
        Ty = np.zeros(Nr)            # shear force in y
        Tz = np.zeros(Nr)            # shear force in z
        My = np.zeros(Nr)            # bending moment about y
        Mz = np.zeros(Nr)            # bending moment about z

        # Reset kinematic arrays for this V0
        rota_angle_y = np.zeros(Nr)  # rotation about y
        rota_angle_z = np.zeros(Nr)  # rotation about z
        uy = np.zeros(Nr)            # edgewise deflection
        uz = np.zeros(Nr)            # flapwise deflection

        # -------- INTERNAL FORCES (Ty, Tz, My, Mz) --------
        for j in range(Nr - 1, 0, -1):

            dx = r_range[j] - r_range[j - 1]

            py_i  = pt_dist[j]
            py_im = pt_dist[j - 1]
            pz_i  = pn_dist[j]
            pz_im = pn_dist[j - 1]

            # Shear forces
            Ty[j - 1] = Ty[j] + 0.5 * (py_i + py_im) * dx
            Tz[j - 1] = Tz[j] + 0.5 * (pz_i + pz_im) * dx

            # Bending moments
            My[j - 1] = My[j] - Tz[j] * dx - (1/6 * pz_im + 1/3 * pz_i) * dx**2
            Mz[j - 1] = Mz[j] + Ty[j] * dx + (1/6 * py_im + 1/3 * py_i) * dx**2

        # -------- CURVATURE (ky, kz) --------
        ky = np.zeros(Nr)
        kz = np.zeros(Nr)

        for l in range(Nr - 1):

            blade_psi = np.deg2rad(pitch_struc[l] + twist_struc[l] + theta)

            # principal-axis bending moments
            M1 = My[l] * np.cos(blade_psi) - Mz[l] * np.sin(blade_psi)
            M2 = My[l] * np.sin(blade_psi) + Mz[l] * np.cos(blade_psi)

            # principal curvatures
            k1 = M1 / EI_1[l]
            k2 = M2 / EI_2[l]

            # convert curvatures back to global coordinates
            ky[l] =  k1 * np.cos(blade_psi) + k2 * np.sin(blade_psi)
            kz[l] = -k1 * np.sin(blade_psi) + k2 * np.cos(blade_psi)

        # -------- DEFLECTION AND ROTATION INTEGRATION --------
        for m in range(Nr - 1):

            dx = r_range[m + 1] - r_range[m]

            # incremental rotations
            rota_angle_y[m + 1] = rota_angle_y[m] + 0.5 * (ky[m + 1] + ky[m]) * dx
            rota_angle_z[m + 1] = rota_angle_z[m] + 0.5 * (kz[m + 1] + kz[m]) * dx

            # incremental deflections
            uy[m + 1] = uy[m] + rota_angle_z[m] * dx + (1/6 * kz[m + 1] + 1/3 * kz[m]) * dx**2
            uz[m + 1] = uz[m] - rota_angle_y[m] * dx - (1/6 * ky[m + 1] + 1/3 * ky[m]) * dx**2

        # -------- STORE RESULTS FOR THIS WIND SPEED --------
        uy_all[k, :] = uy
        uz_all[k, :] = uz
        rota_y_all[k, :] = rota_angle_y
        rota_z_all[k, :] = rota_angle_z

    return {
        "V0": V0_selected,
        "r": r_range,
        "uy": uy_all,
        "uz": uz_all,
        "theta_y": rota_y_all,
        "theta_z": rota_z_all,
    }


# WEIBULL DISTRIBUTION --- Function to compute Weibull probability density function
def weibull_pdf(V, A_weibull=9.0, k_weibull=1.9):
    V = np.asarray(V, dtype=float)
    pdf = (k_weibull / A_weibull) * (V / A_weibull)**(k_weibull - 1.0) * np.exp(-(V / A_weibull)**k_weibull)
    pdf[V < 0.0] = 0.0
    return pdf


# AEP FROM POWER CURVE --- Function to compute AEP in Wh from power curve and Weibull parameters
def aep_from_power_curve(V, P, A_weibull=9.0, k_weibull=1.9):
    f = weibull_pdf(V, A_weibull, k_weibull)
    AEP_Wh = 8760.0 * np.trapezoid(P * f, V)
    return AEP_Wh


# AEP LOSS DUE TO CUT-OUT REDUCTION --- Function to compute AEP loss due to reduction in cut-out wind speed (used for question 5 in Assignment 1). Outputs AEP at reference and altered cut-out speeds, as well as absolute and percentage loss
def q5_solve(q3_power_dict, A_weibull=9.0, k_weibull=1.9,
             cut_out_ref=25.0, cut_out_alt=20.0, make_plot=True):

    # use Q3 variable names
    V = np.asarray(q3_power_dict["V0"], dtype=float)
    P = np.asarray(q3_power_dict["P_V0"], dtype=float)  # mechanical power vs V0

    # enforce cut-in / cut-out
    P_ref = P.copy(); P_alt = P.copy()
    P_ref[(V < cut_in) | (V > cut_out_ref)] = 0.0
    P_alt[(V < cut_in) | (V > cut_out_alt)] = 0.0

    # AEPs
    AEP_ref_MWh = aep_from_power_curve(V, P_ref, A_weibull, k_weibull) / 1e6
    AEP_alt_MWh = aep_from_power_curve(V, P_alt, A_weibull, k_weibull) / 1e6
    loss_MWh = AEP_ref_MWh - AEP_alt_MWh
    loss_pct = 100.0 * loss_MWh / AEP_ref_MWh if AEP_ref_MWh > 0 else np.nan


    return AEP_ref_MWh, AEP_alt_MWh, loss_MWh, loss_pct



#-------------------------PLOTTER FUNCTIONS---------------------------------


def plot_q1_results(Cp1, CT1, Cp2, CT2,lambdas=LAMBDA_GRID,thetas_deg=THETA_GRID,max1=None, max2=None, show=True, savepath=None):

    L, T = np.meshgrid(lambdas, thetas_deg)
    fig, axs = plt.subplots(2, 2, figsize=(10, 8), constrained_layout=True)

    maps = [("Cp(λ,θp) — Eq.(1)", Cp1),
            ("Cp(λ,θp) — Eq.(2)", Cp2),
            ("CT(λ,θp) — Eq.(1)", CT1),
            ("CT(λ,θp) — Eq.(2)", CT2)]
    for ax, (title, Z) in zip(axs.flat, maps):
        cs = ax.contourf(L, T, Z, levels=18)
        plt.colorbar(cs, ax=ax)
        ax.set_xlabel("λ")
        ax.set_ylabel("θp [deg]")
        ax.set_title(title)

    if max1 is not None:
        Cpmax, lamx, thx = max1
        axs[0,0].plot([lamx],[thx],'kx')
        axs[0,0].annotate(f"Cpmax={Cpmax:.3f}", (lamx, thx))
    if max2 is not None:
        Cpmax, lamx, thx = max2
        axs[0,1].plot([lamx],[thx],'kx')
        axs[0,1].annotate(f"Cpmax={Cpmax:.4f}", (lamx, thx))

    if savepath:
        plt.savefig(savepath, dpi=150)
    if show:
        plt.show()
    plt.close(fig)


def plot_rated(V0, V0_rated, omega_max, omega, Eq="Eq"):

    print(f"V0_rated = {V0_rated:.3f} m/s")
    print(f"ω_max    = {omega_max:.4f} rad/s  (= {omega_max * 60 / (2 * np.pi):.2f} rpm)")

    plt.figure(figsize=(7, 5))
    plt.plot(V0, omega)
    plt.xlabel("Wind speed $V_0$ [m/s]")
    plt.ylabel("Rotor speed $\\omega$ [rad/s]")
    plt.title(rf"$\omega(V_0)$ at $C_{{P,\max}}$ — {Eq}")
    plt.tight_layout()
    plt.show()


def plot_q3_results(q3_a, q3_b, show=True, savepath=None):
    V0_a = q3_a["V0"]; V0_b = q3_b["V0"]
    label_a = q3_a.get("mode", "A"); label_b = q3_b.get("mode", "B")

    plt.figure(figsize=(10, 8))

    # P(V0)
    plt.subplot(3, 2, 1)
    plt.plot(V0_a, q3_a["P_V0"] / 1e6, label=label_a)
    plt.plot(V0_b, q3_b["P_V0"] / 1e6, label=label_b)
    plt.axhline(P_mech / 1e6, ls="--", color="k")
    plt.xlabel("V0 [m/s]"); plt.ylabel("P [MW]"); plt.title("P(V0)"); plt.grid(True); plt.legend()

    # θp(V0)
    plt.subplot(3, 2, 2)
    plt.plot(V0_a, q3_a["theta_V0"], label=label_a)
    plt.plot(V0_b, q3_b["theta_V0"], label=label_b)
    plt.xlabel("V0 [m/s]"); plt.ylabel("θp [deg]"); plt.title("θp(V0)"); plt.grid(True); plt.legend()

    # Thrust T(V0)
    plt.subplot(3, 2, 3)
    plt.plot(V0_a, q3_a["T_V0"] / 1e6, label=label_a)
    plt.plot(V0_b, q3_b["T_V0"] / 1e6, label=label_b)
    plt.xlabel("V0 [m/s]"); plt.ylabel("T [MN]"); plt.title("Thrust T(V0)"); plt.grid(True); plt.legend()

    # Cp(V0)
    plt.subplot(3, 2, 4)
    plt.plot(V0_a, q3_a["Cp_V0"], label=label_a)
    plt.plot(V0_b, q3_b["Cp_V0"], label=label_b)
    plt.xlabel("V0 [m/s]"); plt.ylabel("Cp [-]"); plt.title("Cp(V0)"); plt.grid(True); plt.legend()

    # CT(V0)
    plt.subplot(3, 2, 5)
    plt.plot(V0_a, q3_a["Ct_V0"], label=label_a)
    plt.plot(V0_b, q3_b["Ct_V0"], label=label_b)
    plt.xlabel("V0 [m/s]"); plt.ylabel("CT [-]"); plt.title("CT(V0)"); plt.grid(True); plt.legend()

    plt.tight_layout()
    if savepath:
        plt.savefig(savepath, dpi=150)
    if show:
        plt.show()
    plt.close()


def plot_q4_radial(q4_out, show=True, savepath_prefix=None):
    V0s = q4_out["V0_selected"]; r = q4_out["r"]
    dT = q4_out["dTdr_all"]; dQ = q4_out["dQdr_all"]


    for k, v in enumerate(V0s):
        # --- read ASHES CSV for this V0 (if present) ---
        csv_file = os.path.join(BASE_DIR, f"Ashes_V{int(v)}.csv")
        r_ex = dT_ex = dQ_ex = None
        try:
            df = pd.read_csv(csv_file, header=None).iloc[:, :3].dropna(how="all")
            r_ex  = pd.to_numeric(df.iloc[:, 0], errors="coerce").to_numpy() + 2.8
            dT_ex = 3 * pd.to_numeric(df.iloc[:, 2], errors="coerce").to_numpy()
            dQ_ex = 3 * pd.to_numeric(df.iloc[:, 1], errors="coerce").to_numpy()
        except FileNotFoundError:
            pass

        # --- make a dedicated figure for this wind speed ---
        fig, (axT, axQ) = plt.subplots(2, 1, figsize=(8, 8), constrained_layout=True)

        # ---------- Thrust subplot ----------
        axT.plot(r, dT[k, :], label="model")
        if r_ex is not None:
            mT = np.isfinite(r_ex) & np.isfinite(dT_ex)
            xT, yT = r_ex[mT], dT_ex[mT]
            orderT = np.argsort(xT)
            ptsT, = axT.plot(xT, yT, ".", ms=3, label="_nolegend_")
            axT.plot(xT[orderT], yT[orderT], "-", lw=1.2, color=ptsT.get_color(), label="ashes")
            valsT = [dT[k, :], yT]
        else:
            valsT = [dT[k, :]]

        axT.set_title(f"Thrust vs r — V0 = {v:.0f} m/s")
        axT.set_xlabel("r [m]"); axT.set_ylabel("dT/dr [N/m]")
        axT.grid(True); axT.legend()

        # auto y-scale (per wind speed)
        yT_all = np.concatenate([a[np.isfinite(a)] for a in valsT if a is not None])
        if yT_all.size:
            ymin, ymax = np.min(yT_all), np.max(yT_all)
            span = ymax - ymin
            pad = 0.08 * span if span > 0 else (abs(ymax) * 0.05 + 1.0)
            axT.set_ylim(ymin - pad, ymax + pad)

        # ---------- Torque subplot ----------
        axQ.plot(r, dQ[k, :], label="model")
        if r_ex is not None:
            mQ = np.isfinite(r_ex) & np.isfinite(dQ_ex)
            xQ, yQ = r_ex[mQ], dQ_ex[mQ]
            orderQ = np.argsort(xQ)
            ptsQ, = axQ.plot(xQ, yQ, ".", ms=3, label="_nolegend_")
            axQ.plot(xQ[orderQ], yQ[orderQ], "-", lw=1.2, color=ptsQ.get_color(), label="ashes")
            valsQ = [dQ[k, :], yQ]
        else:
            valsQ = [dQ[k, :]]

        axQ.set_title(f"Torque vs r — V0 = {v:.0f} m/s")
        axQ.set_xlabel("r [m]"); axQ.set_ylabel("dQ/dr [N]")
        axQ.grid(True); axQ.legend()

        # auto y-scale (per wind speed)
        yQ_all = np.concatenate([a[np.isfinite(a)] for a in valsQ if a is not None])
        if yQ_all.size:
            ymin, ymax = np.min(yQ_all), np.max(yQ_all)
            span = ymax - ymin
            pad = 0.08 * span if span > 0 else (abs(ymax) * 0.05 + 1.0)
            axQ.set_ylim(ymin - pad, ymax + pad)

        if show:
            plt.show()
        plt.close(fig)


    if show: plt.show()
    plt.close(fig)

## DEFLECTIONS PLOTTER

def plot_deflections_and_angles(r_range, V0_selected,uy_all, uz_all, rota_y_all, rota_z_all):

    r_range = np.asarray(r_range)
    V0_selected = np.asarray(V0_selected)
    uy_all = np.asarray(uy_all)
    uz_all = np.asarray(uz_all)
    rota_y_all = np.asarray(rota_y_all)
    rota_z_all = np.asarray(rota_z_all)

    nV, N = uy_all.shape
    if len(r_range) != N:
        raise ValueError("r_range length must match second dimension of deflection arrays")
    if V0_selected.shape[0] != nV:
        raise ValueError("V0_selected length must match first dimension of deflection arrays")

    # convert angles to degrees
    rota_y_deg = np.degrees(rota_y_all)
    rota_z_deg = np.degrees(rota_z_all)

    fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex=True)
    ax_ang_y, ax_ang_z = axes[0, 0], axes[0, 1]
    ax_def_y, ax_def_z = axes[1, 0], axes[1, 1]

    # --- rotation about y ---
    for k in range(nV):
        ax_ang_y.plot(r_range, rota_y_deg[k, :],
                      label=f"V0 = {V0_selected[k]:.0f} m/s")
    ax_ang_y.set_ylabel(r"Rotation about $y$ [deg]")
    ax_ang_y.set_title("Blade rotation (edgewise)")
    ax_ang_y.grid(True)
    ax_ang_y.legend()

    # --- rotation about z ---
    for k in range(nV):
        ax_ang_z.plot(r_range, rota_z_deg[k, :],
                      label=f"V0 = {V0_selected[k]:.0f} m/s")
    ax_ang_z.set_ylabel(r"Rotation about $z$ [deg]")
    ax_ang_z.set_title("Blade rotation (flapwise)")
    ax_ang_z.grid(True)
    ax_ang_z.legend()

    # --- deflection in y ---
    for k in range(nV):
        ax_def_y.plot(r_range, uy_all[k, :],
                      label=f"V0 = {V0_selected[k]:.0f} m/s")
    ax_def_y.set_xlabel("Radius r [m]")
    ax_def_y.set_ylabel(r"Deflection $u_y$ [m]")
    ax_def_y.set_title("Edgewise deflection")
    ax_def_y.grid(True)
    ax_def_y.legend()

    # --- deflection in z ---
    for k in range(nV):
        ax_def_z.plot(r_range, uz_all[k, :],
                      label=f"V0 = {V0_selected[k]:.0f} m/s")
    ax_def_z.set_xlabel("Radius r [m]")
    ax_def_z.set_ylabel(r"Deflection $u_z$ [m]")
    ax_def_z.set_title("Flapwise deflection")
    ax_def_z.grid(True)
    ax_def_z.legend()

    fig.tight_layout()
    plt.show()


def main():

    print("\n==============================")
    print("       BEM MAIN PROGRAM       ")
    print("==============================\n")

    # ---------------------------------------
    # 1. Ask for momentum equation
    # ---------------------------------------
    print("Select momentum equation:")
    print("  1) Eq. 1 (classical BEM)")
    print("  2) Eq. 2 (Glauert high-induction)")
    eq_choice = int(input("Enter option (1 or 2): ").strip())

    if eq_choice == 1:
        eq_sel = "eq1"
    elif eq_choice == 2:
        eq_sel = "eq2"
    else:
        raise ValueError("Invalid choice. Enter 1 or 2.")


    # ---------------------------------------
    # 2. Ask for pitching method (Q3 onward)
    # ---------------------------------------
    print("\nSelect pitching method:")
    print("  1) Feather")
    print("  2) Stall")
    pitch_choice = int(input("Enter option (1 or 2): ").strip())

    if pitch_choice == 1:
        pitch_method = "feather"
    elif pitch_choice == 2:
        pitch_method = "stall"
    else:
        raise ValueError("Invalid choice. Enter 1 or 2.")


    # ---------------------------------------
    # 3. Ask for output mode
    # ---------------------------------------
    print("\nSelect output mode:")
    print("  1) Plot results")
    print("  2) Print numerical values")
    print("  3) Both")
    mode_choice = int(input("Enter option (1, 2, or 3): ").strip())

    if mode_choice == 1:
        out_mode = "plot"
    elif mode_choice == 2:
        out_mode = "print"
    elif mode_choice == 3:
        out_mode = "both"
    else:
        raise ValueError("Invalid option.")


    # =======================================
    # Q1 — Compute Cp and CT maps
    # =======================================
    print("\n--- Q1: Computing Cp(λ,θ) maps ---")
    Cp1, CT1, Cp1max, lam1, th1 = compute_finalvalues(eq="eq1")
    Cp2, CT2, Cp2max, lam2, th2 = compute_finalvalues(eq="eq2")

    # Select which optimum to use for Q2+
    if eq_sel == "eq1":
        Cp_max = Cp1max; lambda_star = lam1; theta_star = th1
    else:
        Cp_max = Cp2max; lambda_star = lam2; theta_star = th2

    if out_mode in ("print", "both"):
        print(f"\nEq.1 → Cp_max={Cp1max:.4f}, λ*={lam1:.3f}, θ*={th1:.3f} deg")
        print(f"Eq.2 → Cp_max={Cp2max:.4f}, λ*={lam2:.3f}, θ*={th2:.3f} deg")

    if out_mode in ("plot", "both"):
        plot_q1_results(Cp1, CT1, Cp2, CT2,
                        lambdas=LAMBDA_GRID,
                        thetas_deg=THETA_GRID,
                        max1=(Cp1max, lam1, th1),
                        max2=(Cp2max, lam2, th2))


    # =======================================
    # Q2 — Rated conditions
    # =======================================
    print("\n--- Q2: Rated conditions ---")
    V0, V0_rated, omega_max, omega, omega_unc = compute_rated(
        Cp_max, lambda_star, A, Eq=eq_sel)

    if out_mode in ("print", "both"):
        print(f"Rated wind speed  : {V0_rated:.3f} m/s")
        print(f"Maximum rotor ω   : {omega_max:.4f} rad/s")

    if out_mode in ("plot", "both"):
        V0 = np.linspace(cut_in, cut_out, 200)
        plot_rated(V0, V0_rated, omega_max, omega, Eq=eq_sel)


    # =======================================
    # Q3 — Pitch control curve
    # =======================================
    print("\n--- Q3: Pitch schedule ---")

    q3_res = pitch_control(lambda_star, V0_rated, theta_star,
                           omega_max, eq=eq_sel, mode=pitch_method)

    if out_mode in ("plot", "both"):
        # For consistency also show "stall" schedule
        q3_other = pitch_control(lambda_star, V0_rated, theta_star,
                                 omega_max, eq=eq_sel,
                                 mode=("stall" if pitch_method=="feather" else "feather"))
        plot_q3_results(q3_res, q3_other)

    if out_mode in ("print", "both"):
        print("Pitch schedule computed.")


    # =======================================
    # Q4 — Spanwise loads
    # =======================================
    print("\n--- Q4: Spanwise loads ---")
    q4 = loads_selected(lambda_star, theta_star, V0_rated, omega_max, eq=eq_sel)

    if out_mode in ("print", "both"):
        print("\nV0 | Pitch | Omega | Thrust [MN] | Torque [MNm]")
        for V, T, Q, om, th in zip(q4["V0_selected"],
                                   q4["T_all"]/1e6,
                                   q4["Q_all"]/1e6,
                                   q4["omega_selected"],
                                   q4["pitch_selected"]):
            print(f"{V:4.0f} | {th:6.2f} | {om:7.3f} | {T:10.4f} | {Q:10.4f}")

    if out_mode in ("plot", "both"):
        plot_q4_radial(q4)


    # =======================================
    # DEFLECTIONS — Static blade bending
    # =======================================
    print("\n--- DEFLECTIONS: Static blade deflection ---")
    defl = deflection_calc(lambda_star, theta_star, V0_rated, omega_max,
                           pitch_struc, mass_dist, EI_1, EI_2, twist_struc,
                           eq=eq_sel)

    if out_mode in ("print", "both"):
        print("\nV0 | uy_tip [m] | uz_tip [m]")
        for i, V in enumerate(defl["V0"]):
            uy_tip = defl["uy"][i, -1]
            uz_tip = defl["uz"][i, -1]
            print(f"{V:4.0f} | {uy_tip:9.4f} | {uz_tip:9.4f}")

    if out_mode in ("plot", "both"):
        plot_deflections_and_angles(defl["r"], defl["V0"],
                                    defl["uy"], defl["uz"],
                                    defl["theta_y"], defl["theta_z"])


    # =======================================
    # Q5 — Annual Energy Production
    # =======================================
    print("\n--- Q5: AEP calculation ---")
    AEP_25, AEP_20, loss, pct = q5_solve(q3_res)

    if out_mode in ("print", "both"):
        print(f"AEP @ 25 m/s cut-out : {AEP_25:.1f} MWh")
        print(f"AEP @ 20 m/s cut-out : {AEP_20:.1f} MWh")
        print(f"Loss                 : {loss:.1f} MWh ({pct:.2f}%)")

    print("\n=== END OF PROGRAM ===\n")



if __name__ == "__main__":
    main()




       BEM MAIN PROGRAM       

Select momentum equation:
  1) Eq. 1 (classical BEM)
  2) Eq. 2 (Glauert high-induction)

Select pitching method:
  1) Feather
  2) Stall

Select output mode:
  1) Plot results
  2) Print numerical values
  3) Both

--- Q1: Computing Cp(λ,θ) maps ---

Eq.1 → Cp_max=0.4663, λ*=7.931, θ*=-0.103 deg
Eq.2 → Cp_max=0.4739, λ*=8.103, θ*=-0.345 deg

--- Q2: Rated conditions ---
Rated wind speed  : 11.425 m/s
Maximum rotor ω   : 1.0162 rad/s

--- Q3: Pitch schedule ---
Pitch schedule computed.

--- Q4: Spanwise loads ---

V0 | Pitch | Omega | Thrust [MN] | Torque [MNm]
   5 |  -0.10 |   0.445 |     0.3218 |     2.0053
   9 |  -0.10 |   0.800 |     1.0426 |     6.4973
  11 |  -0.10 |   0.978 |     1.5575 |     9.7059
  20 |  17.24 |   1.016 |     0.6736 |    10.4375

--- DEFLECTIONS: Static blade deflection ---

V0 | uy_tip [m] | uz_tip [m]
   6 |    0.2766 |    2.8753
  11 |    0.9297 |    9.6643
  20 |    0.6946 |    1.8422

--- Q5: AEP calculation ---
AEP @ 2